In [2]:
from keras.layers import Input, Dense
from keras.models import Model

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(30000,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(30000, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

Using TensorFlow backend.


In [3]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

In [4]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [5]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [ ]:
import pandas as pd
import numpy as np
import skimage
from sklearn.model_selection import train_test_split

img_folder = r'C:\Users\Kamil Ogrodowski\Desktop\Obrazowanie\218447-medical_imaging-788b05f861ac\dataset'
img_width = 100
img_height = 100

# Read data
data = pd.read_csv(r'C:\Users\Kamil Ogrodowski\Desktop\Obrazowanie\218447-medical_imaging-788b05f861ac\dataset\GTruth.csv')

# Split to train_data, val_data, test_data
train_data, test_data = train_test_split(data)

def read_img(fileid):
    """
    Read and resize img, adjust channels. 
    Caution: This function is not independent, it uses global vars: img_folder, img_channels
    @param file: file id, int
    """
    img = skimage.io.imread(img_folder +'\\'+ str(fileid) + '.jpeg')
    img = skimage.transform.resize(img, (img_width, img_height), mode='reflect')
    # A few image are grey, duplicate them for to have 3 alpha channels.
    if(len(img.shape) < 3):
        img = np.dstack([img, img, img])
    return img
                        
# Train data
train_X = np.stack(train_data['Id'].apply(read_img))
test_X = np.stack(test_data['Id'].apply(read_img))

c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [ ]:
train_X = train_X.astype('float32') / 255.
test_X = test_X.astype('float32') / 255.
train_X = train_X.reshape((len(train_X), np.prod(train_X.shape[1:])))
test_X = test_X.reshape((len(test_X), np.prod(test_X.shape[1:])))
print(train_X.shape)
print(test_X.shape)

(4392, 30000)
(1464, 30000)


In [ ]:
autoencoder.fit(train_X, train_X,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(test_X, test_X))

Train on 4392 samples, validate on 1464 samples
Epoch 1/50
4392/4392 [==============================] - 12s 3ms/step - loss: 0.6932 - val_loss: 0.6932
Epoch 2/50
4392/4392 [==============================] - 10s 2ms/step - loss: 0.6931 - val_loss: 0.6929
Epoch 3/50
4392/4392 [==============================] - 10s 2ms/step - loss: 0.6928 - val_loss: 0.6927
Epoch 4/50
4392/4392 [==============================] - 10s 2ms/step - loss: 0.6927 - val_loss: 0.6926
Epoch 5/50
4392/4392 [==============================] - 11s 2ms/step - loss: 0.6926 - val_loss: 0.6925
Epoch 6/50
4392/4392 [==============================] - 11s 2ms/step - loss: 0.6923 - val_loss: 0.6922
Epoch 7/50
4392/4392 [==============================] - 10s 2ms/step - loss: 0.6921 - val_loss: 0.6921
Epoch 8/50
4392/4392 [==============================] - 10s 2ms/step - loss: 0.6920 - val_loss: 0.6919
Epoch 9/50
4392/4392 [==============================] - 10s 2ms/step - loss: 0.6918 - val_loss: 0.6916
Epoch 10/50
4392/4392 [==

In [ ]:
# encode and decode some digits
# note that we take them from the *test* set
# print(decoder)
# print(vars(decoder))
# print(dir(decoder))
print(decoder.inputs[0].shape)
encoded_imgs = encoder.predict(test_X)
decoded_imgs = decoder.predict(encoded_imgs)

In [ ]:
# use Matplotlib (don't ask)
import matplotlib.pyplot as plt
from scipy.misc import imsave

# n = 1  # how many digits we will display
# plt.figure(figsize=(20, 4))
# for i in range(n):
#     # display original
#     print(test_X[i])
#     ax = plt.subplot(2, n, i + 1)
#     plt.imshow(test_X[i].reshape(100, 100, 3))
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)

#     # display reconstruction
#     ax = plt.subplot(2, n, i + 1 + n)
#     plt.imshow(decoded_imgs[i].reshape(100, 100, 3))
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)
# plt.show()

imsave('original.png', test_X[0].reshape(100, 100, 3))
imsave('decoded.png', decoded_imgs[0].reshape(100, 100, 3))